In [14]:
import json, requests
import pandas as pd
import numpy as np
import io
import random
import time

from random import sample
import warnings
warnings.filterwarnings('ignore')

In [15]:
from transform_functions import stock_data_pull, fill_blanks, feature_create

In [16]:
# import cml.data_v1 as cmldata

# #CONNECTION_NAME = "loki-hive-vw"
# CONNECTION_NAME = "loki-impala-vw"
# conn = cmldata.get_connection(CONNECTION_NAME)

# ## Sample Usage to get pandas data frame
# EXAMPLE_SQL_QUERY = "show databases"
# dataframe = conn.get_pandas_dataframe(EXAMPLE_SQL_QUERY)
# print(dataframe)


# not able to connect..
# KeyError: 'No password specified. Please edit WORKLOAD_PASSWORD environment variable under User Settings > Environment Variables'

In [17]:
stock_total = pd.read_csv('query-impala-319.csv')

In [18]:
stock_total.columns

Index(['symbol', 'symbol_time', 'open', 'high', 'low', 'close', 'volume',
       'last_refreshed', 'information'],
      dtype='object')

In [19]:
stock_total = stock_total[['symbol','symbol_time','close', 'volume']]
stock_total.head()

,symbol,symbol_time,close,volume
0,AAL,2023-01-10 19:10:00,15.2000,895
1,AAL,2023-01-10 18:44:00,15.2000,123
2,AAL,2023-01-10 18:28:00,15.1700,322
3,AAL,2023-01-10 19:24:00,15.2200,3611
4,AAL,2023-01-10 19:52:00,15.2091,130


**Define parameters**

In [20]:
date_range = stock_total.symbol_time.drop_duplicates()
date_range = pd.DataFrame(date_range)
stock_ready = fill_blanks(stock_total,'symbol',date_range,'symbol_time');

In [21]:
window = 20  
features = ['return', 'vol', 'mom', 'sma', 'min', 'max']  
lags = 6

**Feature Create**

In [22]:
tckr_list = stock_ready.symbol.unique().tolist()

In [23]:
# call function to genereate all new variables
#def feature_create(df_in,tckr_list, window,lags,features,clip=False,tckr='Name',date='Datetime',close='Close):

stk_rdy_2, cols =  feature_create(stock_ready,tckr_list, window,lags,features,clip=True,tckr='symbol',date='symbol_time',close='close')

all good, all data contains 100 observations
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80


In [24]:
stk_rdy_2.shape

(90, 47)

In [25]:
stk_rdy_2.head()

,symbol,symbol_time,close,volume,return,vol,mom,sma,min,max,...,min_lag_4,min_lag_5,min_lag_6,max_lag_1,max_lag_2,max_lag_3,max_lag_4,max_lag_5,max_lag_6,direction
0,AAL,2023-01-10 20:00:00,15.210000,547.0,0.000000,0.000671,-1.0,15.212465,15.200000,15.240000,...,15.200000,15.200000,15.200000,15.240000,15.240000,15.240000,15.240000,15.240000,15.240000,-1
1,AAPL,2023-01-10 20:00:00,130.970001,1925.0,-0.000076,0.000306,1.0,130.885500,130.759995,130.979996,...,130.759995,130.759995,130.759995,130.979996,130.970001,130.919998,130.919998,130.919998,130.919998,-1
2,ALK,2023-01-10 20:00:00,47.299999,501.0,0.000000,0.000000,0.0,47.299999,47.299999,47.299999,...,47.299999,47.299999,47.299999,47.299999,47.299999,47.299999,47.299999,47.299999,47.299999,-1
3,AMZN,2023-01-10 20:00:00,89.999901,7216.0,0.000221,0.000478,1.0,89.968490,89.870003,89.999901,...,89.870003,89.870003,89.940002,89.999901,89.999901,89.999901,89.999901,89.999901,89.999901,1
4,BAX,2023-01-10 20:00:00,45.740002,122.0,0.000000,0.000000,0.0,45.740002,45.740002,45.740002,...,45.740002,45.740002,45.740002,45.740002,45.740002,45.740002,45.740002,45.740002,45.740002,-1


In [26]:
stk_rdy_2.columns

Index(['symbol', 'symbol_time', 'close', 'volume', 'return', 'vol', 'mom',
       'sma', 'min', 'max', 'return_lag_1', 'return_lag_2', 'return_lag_3',
       'return_lag_4', 'return_lag_5', 'return_lag_6', 'vol_lag_1',
       'vol_lag_2', 'vol_lag_3', 'vol_lag_4', 'vol_lag_5', 'vol_lag_6',
       'mom_lag_1', 'mom_lag_2', 'mom_lag_3', 'mom_lag_4', 'mom_lag_5',
       'mom_lag_6', 'sma_lag_1', 'sma_lag_2', 'sma_lag_3', 'sma_lag_4',
       'sma_lag_5', 'sma_lag_6', 'min_lag_1', 'min_lag_2', 'min_lag_3',
       'min_lag_4', 'min_lag_5', 'min_lag_6', 'max_lag_1', 'max_lag_2',
       'max_lag_3', 'max_lag_4', 'max_lag_5', 'max_lag_6', 'direction'],
      dtype='object')

In [27]:
len(stk_rdy_2.symbol.unique())

90

In [28]:
input_ds = stk_rdy_2[cols]

In [29]:
input_ds.columns

Index(['return_lag_1', 'return_lag_2', 'return_lag_3', 'return_lag_4',
       'return_lag_5', 'return_lag_6', 'vol_lag_1', 'vol_lag_2', 'vol_lag_3',
       'vol_lag_4', 'vol_lag_5', 'vol_lag_6', 'mom_lag_1', 'mom_lag_2',
       'mom_lag_3', 'mom_lag_4', 'mom_lag_5', 'mom_lag_6', 'sma_lag_1',
       'sma_lag_2', 'sma_lag_3', 'sma_lag_4', 'sma_lag_5', 'sma_lag_6',
       'min_lag_1', 'min_lag_2', 'min_lag_3', 'min_lag_4', 'min_lag_5',
       'min_lag_6', 'max_lag_1', 'max_lag_2', 'max_lag_3', 'max_lag_4',
       'max_lag_5', 'max_lag_6'],
      dtype='object')

In [30]:
input_ds.shape

(90, 36)

**Test existing model**

In [31]:
input_ds# Model - new model- 
model_url = 'https://modelservice.ml-6e89be47-314.loki-cdp.a465-9q4k.cloudera.site/model'
accessKey = 'm11sdyxpwyfdnbe1cf45keu9q260j765'


In [32]:
# build input for api into dictionary w/ values as a list
input_data_mult = dict()
for col in input_ds.columns:
    input_data_mult[col] = input_ds[col].to_list()    
    
# build embedded dictionary step 1    
request_dict = {"request":input_data_mult}

# access key will be end point specific
BackDict = {"accessKey":accessKey}
BackDict.update(request_dict)
request_dict=BackDict

In [13]:
request_dict

NameError: name 'request_dict' is not defined

In [35]:
r9 = requests.post(model_url, data=json.dumps(request_dict),headers={'Content-Type': 'application/json'})

In [36]:
r9.json()

{'success': True,
 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:61120f1b-0e6a-483d-a73d-6594c1f7ae6f/1dffefd1-084b-43f0-acf9-eaf1b569f00c',
  'prediction': [-1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   1,
   -1,
   1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   1,
   -1,
   1,
   -1,
   -1,
   1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   -1,
   1,
   -1],
  'uuid': 'a2f792af-3e34-4019-93fe-39fa8e9e5f9b'},
 'ReplicaID': 'rf-direction-3-7-11-7b5957fc5d-m58vp',
 'Size': 520,
 'StatusCode': 200}

### Testing full model serve function

In [39]:
from joblib import dump, load

In [62]:
def mkt_movement(args):
    
    window = 20  
    features = ['return', 'vol', 'mom', 'sma', 'min', 'max']  
    lags = 6
    
    # read in data direction from kudu (in json format)
    data = pd.DataFrame.from_dict(args)
    data = data[exp_cols]
    
    # fill missing data
    date_range = stock_total.symbol_time.drop_duplicates()
    date_range = pd.DataFrame(date_range)
    stock_ready = fill_blanks(data,'symbol',date_range,'symbol_time');
    
    # transform dataset
    tckr_list = stock_ready.symbol.unique().tolist()
    stk_rdy_2, cols =  feature_create(stock_ready,tckr_list, window,lags,features,clip=True,tckr='symbol',date='symbol_time',close='close')
    
    # predict
    pipe_rf = load("saved_models/pipe_rf.joblib")
    prediction = pipe_rf.predict(stk_rdy_2[cols])
    # output prep
    pred_time_list = [stk_rdy_2.symbol_time.max()]*len(stk_rdy_2)
    output = {'symbol':stk_rdy_2.symbol.tolist(),'pred_time':pred_time_list,'prediction':prediction}
    out_df = pd.DataFrame.from_dict(output)
    
    return out_df

90

In [54]:
exp_cols = ['symbol','symbol_time','close', 'volume']

In [55]:
kudu_input = pd.read_csv('query-impala-319.csv')

In [56]:
input_dict = dict()
for col in kudu_input.columns:
    input_dict[col] = kudu_input[col].to_list()        

In [60]:
pred,stk_rdy = mkt_movement(input_dict)

all good, all data contains 100 observations
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80
size of temp is after clipping 1
size of temp is 80


In [61]:
pred

,symbol,pred_time,prediction
0,AAL,2023-01-10 20:00:00,-1
1,AAPL,2023-01-10 20:00:00,-1
2,ALK,2023-01-10 20:00:00,-1
3,AMZN,2023-01-10 20:00:00,-1
4,BAX,2023-01-10 20:00:00,-1
...,...,...,...
85,WBD,2023-01-10 20:00:00,-1
86,WDC,2023-01-10 20:00:00,-1
87,WYNN,2023-01-10 20:00:00,-1
88,XOM,2023-01-10 20:00:00,1


In [52]:
pred.dtypes

symbol        object
pred_time     object
prediction     int64
dtype: object

In [58]:
stk_rdy.head()

,symbol,symbol_time,close,volume,return,vol,mom,sma,min,max,...,min_lag_4,min_lag_5,min_lag_6,max_lag_1,max_lag_2,max_lag_3,max_lag_4,max_lag_5,max_lag_6,direction
0,AAL,2023-01-10 20:00:00,15.210000,547.0,0.000000,0.000671,-1.0,15.212465,15.200000,15.240000,...,15.200000,15.200000,15.200000,15.240000,15.240000,15.240000,15.240000,15.240000,15.240000,-1
1,AAPL,2023-01-10 20:00:00,130.970001,1925.0,-0.000076,0.000306,1.0,130.885500,130.759995,130.979996,...,130.759995,130.759995,130.759995,130.979996,130.970001,130.919998,130.919998,130.919998,130.919998,-1
2,ALK,2023-01-10 20:00:00,47.299999,501.0,0.000000,0.000000,0.0,47.299999,47.299999,47.299999,...,47.299999,47.299999,47.299999,47.299999,47.299999,47.299999,47.299999,47.299999,47.299999,-1
3,AMZN,2023-01-10 20:00:00,89.999901,7216.0,0.000221,0.000478,1.0,89.968490,89.870003,89.999901,...,89.870003,89.870003,89.940002,89.999901,89.999901,89.999901,89.999901,89.999901,89.999901,1
4,BAX,2023-01-10 20:00:00,45.740002,122.0,0.000000,0.000000,0.0,45.740002,45.740002,45.740002,...,45.740002,45.740002,45.740002,45.740002,45.740002,45.740002,45.740002,45.740002,45.740002,-1
